# Chest X-ray Classification

This notebook presents the chest X-ray classification use-case on NIH chest X-ray dataset which supports prediction and evaluation of models provided by `torchxrayvision` package.

In [ ]:
import os

import numpy as np
import pandas as pd
from datasets import Dataset
from datasets.features import Image
from monai.transforms import AddChanneld, Compose, Lambdad, Resized, SqueezeDimd

from cyclops.data.slicer import SliceSpec
from cyclops.evaluate.metrics import AUROC, MetricCollection, create_metric
from cyclops.models.catalog import create_model
from cyclops.models.utils import get_device
from cyclops.tasks.cxr_classification import CXRClassificationTask
from cyclops.utils.file import join

## Constants

In [ ]:
NIHCXR_DIR = "/mnt/data/clinical_datasets/NIHCXR"
FEATURE_COL = "image"

## Data Loading and Preprocessing

Loading the NIH dataset as a pandas dataframe.

In [ ]:
test_df = pd.read_csv(
    join(NIHCXR_DIR, "test_list.txt"), header=None, names=["Image Index"],
)

# select only the images in the test list
df = pd.read_csv(join(NIHCXR_DIR, "Data_Entry_2017.csv"))
df.dropna(how="all", axis="columns", inplace=True)  # drop empty columns
df = df[df["Image Index"].isin(test_df["Image Index"])]

In the preprocessing step the labels are added to the dataframe with a one-hot encoding representation. The dataframe is converted to Hugging Face dataset, which will be later used to load and preprocess the images.

In [ ]:
def nihcxr_preprocess(df: pd.DataFrame, nihcxr_dir: str) -> pd.DataFrame:
    """Preprocess NIHCXR dataframe.

    Add a column with the path to the image and create one-hot encoded pathogies
    from Finding Labels column.

    Args:
    ----
        df (pd.DataFrame): NIHCXR dataframe.

    Returns:
    -------
        pd.DataFrame: pre-processed NIHCXR dataframe.
    """
    # Add path column
    df[FEATURE_COL] = df["Image Index"].apply(
        lambda x: os.path.join(nihcxr_dir, "images", x),
    )

    # Create one-hot encoded pathologies
    pathologies = df["Finding Labels"].str.get_dummies(sep="|")
    # Add one-hot encoded pathologies to dataframe
    df = pd.concat([df, pathologies], axis=1)

    return df, list(pathologies.columns)

In [ ]:
df, data_pathologies = nihcxr_preprocess(df, NIHCXR_DIR)
df

In [ ]:
data_pathologies

Creating a Hugging Face Dataset from the dataframe.

In [ ]:
# create a Dataset object
nih_ds = Dataset.from_pandas(df, preserve_index=False)
nih_ds = nih_ds.cast_column(FEATURE_COL, Image())
nih_ds

## Model Creation

The CyclOps Model API is used to wrapp the models from `torchxrayvision`. The configuration of the model is based on the corresponding config files, which include the necessary parameters for instantiating the models including the weights of pretrained models.

In [ ]:
densenet_name = "densenet"
densenet = create_model(densenet_name)
# initalize the model to load weights
densenet.initialize()

In [ ]:
resnet_name = "resnet"
resnet = create_model(resnet_name)
resnet.initialize()

Note that the labels in the dataset and those that are learned by the pretrained models are not necessary the same. Later on the common labels will be used for model evaluation.

In [ ]:
model_pathologies = densenet.model_.pathologies
model_pathologies

In [ ]:
resnet.model_.pathologies == model_pathologies

In [ ]:
common_pathologies = list(set(model_pathologies) & set(data_pathologies))
common_pathologies

In [ ]:
data_extra_labels = [
    label for label in data_pathologies if label not in common_pathologies
]
data_extra_labels

In [ ]:
model_extra_labels = [
    label for label in model_pathologies if label not in common_pathologies
]
model_extra_labels

## Chest X-Ray Classification Task

The CyclOps Task API is used to create a CXR Classification Task based on the available models and dataset. The task can contain multiple models that can be used for prediction individually. This is particularly useful when comparing the performance of multiple models during the evaluation step.

In [ ]:
cxr_task = CXRClassificationTask(
    {densenet_name: densenet, resnet_name: resnet},
    task_features=[FEATURE_COL],
    task_target=model_pathologies,
)
cxr_task.list_models()

### Prediction

For prediction, the task object allows for numpy arrays and Hugging Face Datasets as input data features.

When using a Hugging Face dataset as the input, you have the option to obtain the entire dataset with the added prediction column as the output of the predict method. This is particularly useful when dealing with large datasets that cannot fit into memory or when batched prediction is desired.

In [ ]:
get_device()

Transforms from `MONAI` are used to be applied to the image data.

In [ ]:
transforms = Compose(
    [
        AddChanneld(keys=(FEATURE_COL,)),
        Resized(keys=(FEATURE_COL,), spatial_size=(1, 224, 224)),
        Lambdad(keys=(FEATURE_COL), func=lambda x: ((2 * (x / 255.0)) - 1.0) * 1024),
        SqueezeDimd(keys=(FEATURE_COL), dim=1),
    ],
)

In [ ]:
ds = cxr_task.predict(
    nih_ds,
    model_name=densenet_name,
    transforms=transforms,
    prediction_column_prefix="predictions",
    only_predictions=False,
)
ds

In [ ]:
preds = cxr_task.predict(
    nih_ds, model_name=resnet_name, transforms=transforms, only_predictions=True,
)

### Evaluation

Evaluation is typically performed on a Hugging Face dataset. To evaluate the models, you can also provide a slice specification on the meta data.

In addition to the dataset and slice specification, you need to specify the desired evaluation metrics. This can be done by providing a MetricCollection object, a list of metrics, or metric names.


In [ ]:
# define the slices
spec_list = [
    {"Patient Gender": {"value": "M"}},
    {"Patient Gender": {"value": "F"}},
    #     {"Patient Age": {"min_value": 25, "max_value": 40}},
    {"Patient Age": {"min_value": 65}},
    {"View Position": {"value": "PA"}},
]

# create the slice functions
slice_spec = SliceSpec(spec_list=spec_list)

In [ ]:
# define the metrics

metric_names = ["accuracy", "precision", "recall", "f1_score"]
metrics = [
    create_metric(metric_name, task="multilabel", num_labels=len(model_pathologies))
    for metric_name in metric_names
]
auroc = AUROC(
    task="multilabel",
    thresholds=np.arange(0, 1, 0.01),
    num_labels=len(model_pathologies),
)
metrics.append(auroc)
metric_collection = MetricCollection(metrics)

In [ ]:
results, dataset_with_preds = cxr_task.evaluate(
    nih_ds,
    metric_collection,
    transforms=transforms,
    batch_size=128,
    prediction_column_prefix="preds",
    slice_spec=slice_spec,
    remove_columns=[FEATURE_COL],
)

In [ ]:
results[densenet_name]

In [ ]:
results[resnet_name]